# Joining User Data
_Laste Updated Monday Jul. 7th By Jing_   
This .ipynb file joins user data stored in:  
- auth_user  
- auth_user_profille  
- student_languageproficiency  
- student_courseenrollment  

where:  
- The `student_profile_id` in `student_languageproficiency` is the `id` field in `auth_user_profile`;  
- `auth_user_profille` and `student_courseenrollment` share common field `user_id`, which is the `id` field in `auth_user`  

*Note:*  
`student_anonymoususerid` is also user data but I didn't find out how could it be relevant. Needs further investigation.

In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

In [2]:
"""
We already read all sql files into the clean directory "sqldir" using transfer_sql 
"""
sqldir = "/Users/Bianbian/Documents/Work/edX/pydata/" # Directory that keeps all .p data
course1 = "ColumbiaX-DS101X-1T2016-" # First part: might change according to course/T
course2 ="-prod-analytics.p" # Latter part: fixed
#Define load function
def open_sql(name):
    with open (sqldir + course1 + name + course2) as f:
        return pickle.load(f)
    
user = open_sql("auth_user")
profile = open_sql("auth_userprofile")
# anno = open_sql("student_anonymoususerid")
lang = open_sql("student_languageproficiency")
enrol= open_sql("student_courseenrollment")

left = user
right = pd.merge(profile,enrol,how='left',on='user_id',suffixes=['', '_enroll']) #pr:profile
right.rename(columns={'id':'user_profile_id'},inplace=True)
right = pd.merge(right,lang,how= 'left', on='user_profile_id',suffixes=['', '_language'])
"Index join"
left = user.set_index('id')
right=right.set_index('user_id')
result=left.join(right, lsuffix = '_user', rsuffix= '_pel')
# Now the index is 'user_id' which uniquely identifies any user record
print result.shape
# Removing all-nan columns
result.replace('',np.nan,inplace=True)
result.replace('NULL',np.nan,inplace= True)
result = result.dropna(axis =1, how = "all")
print result.shape

(33955, 44)
(33955, 35)


In [3]:
result.to_csv("full_profile.csv")